<a href="https://colab.research.google.com/github/PranavVaish/UCS654_Topsis_102303194/blob/main/UCS654_Topsis_102303194.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import sys
import pandas as pd
import numpy as np
import os

def check_numeric(df):
    try:
        # Select all columns from index 1 to the end
        data_part = df.iloc[:, 1:]
        # Attempt to convert to float; this will raise an error if non-numeric data exists
        data_part.astype(float)
        return True
    except Exception:
        return False

def topsis(input_file, weights, impacts, result_file):
    try:
        if not os.path.exists(input_file):
            raise FileNotFoundError(f"Error: The file '{input_file}' was not found.")
        df = pd.read_csv(input_file)
        if df.shape[1] < 3:
            raise ValueError("Error: Input file must contain three or more columns.")
        if not check_numeric(df):
            raise ValueError("Error: From 2nd to last columns must contain numeric values only.")
        # Convert weights string "1,1,1,2" to list of floats
        try:
            weight_list = [float(w) for w in weights.split(',')]
        except ValueError:
             raise ValueError("Error: Weights must be numeric and separated by commas.")
        # Convert impacts string "+,+,-,+" to list
        impact_list = impacts.split(',')
        # Dimensions of the numeric part of data
        num_cols = df.shape[1] - 1 # Subtracting the first name/ID column
        # Check parameter counts match
        if len(weight_list) != num_cols or len(impact_list) != num_cols:
            raise ValueError(f"Error: Number of weights ({len(weight_list)}), impacts ({len(impact_list)}), "
                             f"and columns ({num_cols}) must be the same.")
        # Check Impact Symbols
        if not all(i in ['+', '-'] for i in impact_list):
            raise ValueError("Error: Impacts must be either '+' or '-'.")

        # TOPSIS ALGORITHM IMPLEMENTATION

        # 1. Convert data to numpy array for easier calculation
        data = df.iloc[:, 1:].values.astype(float)

        # 2. Normalize the matrix
        # Square root of sum of squares for each column
        rss = np.sqrt(np.sum(data**2, axis=0))
        normalized_data = data / rss

        # 3. Weighted Normalization
        weighted_data = normalized_data * weight_list

        # 4. Ideal Best and Ideal Worst
        ideal_best = []
        ideal_worst = []

        for i in range(num_cols):
            if impact_list[i] == '+':
                ideal_best.append(np.max(weighted_data[:, i]))
                ideal_worst.append(np.min(weighted_data[:, i]))
            else: # Impact is '-'
                ideal_best.append(np.min(weighted_data[:, i]))
                ideal_worst.append(np.max(weighted_data[:, i]))

        ideal_best = np.array(ideal_best)
        ideal_worst = np.array(ideal_worst)

        # 5. Euclidean Distance
        # Distance from Best
        s_plus = np.sqrt(np.sum((weighted_data - ideal_best)**2, axis=1))
        # Distance from Worst
        s_minus = np.sqrt(np.sum((weighted_data - ideal_worst)**2, axis=1))

        # 6. Performance Score
        # Handle division by zero edge case if s_plus + s_minus is 0
        total_dist = s_plus + s_minus
        performance_score = np.divide(s_minus, total_dist, out=np.zeros_like(s_minus), where=total_dist!=0)

        # 7. Append to DataFrame
        df['Topsis Score'] = performance_score

        # 8. Rank (Higher score is better)
        df['Rank'] = df['Topsis Score'].rank(ascending=False).astype(int)

        # 9. Save to output file
        df.to_csv(result_file, index=False)
        print(f"Success: TOPSIS results saved to '{result_file}'")

    except FileNotFoundError as fnf_error:
        print(fnf_error)
    except ValueError as val_error:
        print(val_error)
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def main():
    if len(sys.argv) != 5:
        print("Usage: python topsis.py <input_file> <weights> <impacts> <result_file>")
        sys.exit(1)

The `topsis` function requires four main inputs:

1.  **`input_file`**: A string representing the path to the CSV file containing your decision matrix. The first column should be the criteria names/identifiers, and subsequent columns should be numeric criterion values.
2.  **`weights`**: A string of comma-separated numeric values representing the weights for each criterion. For example, `'1,2,1,3'`.
3.  **`impacts`**: A string of comma-separated `'+'` or `'-'` symbols, indicating whether a higher value is better (`'+'`) or worse (`'-'`) for each criterion. For example, `'+,+,-,+'`.
4.  **`result_file`**: A string representing the desired path for the output CSV file, which will contain the original data along with the calculated TOPSIS score and rank.

In [34]:
# First, let's create a sample input file named 'data.csv'
import pandas as pd

data = {
    'Alternative': ['A1', 'A2', 'A3', 'A4'],
    'Criterion1': [25, 40, 60, 30],
    'Criterion2': [2000, 3000, 2500, 3500],
    'Criterion3': [5, 4, 3, 4],
    'Criterion4': [10000, 8000, 12000, 9000],
    'Criterion5': [7, 6, 8, 5]
}
df_input = pd.DataFrame(data)
df_input.to_csv('data.csv', index=False)

print("Sample 'data.csv' created:")
display(df_input)

Sample 'data.csv' created:


,Alternative,Criterion1,Criterion2,Criterion3,Criterion4,Criterion5
0,A1,25,2000,5,10000,7
1,A2,40,3000,4,8000,6
2,A3,60,2500,3,12000,8
3,A4,30,3500,4,9000,5


In [35]:
# Now, call the topsis function with example inputs
# Make sure the cell defining the `topsis` function has been executed first!

input_filename = 'data.csv'
weights_str = '0.2,0.3,0.1,0.25,0.15' # Example weights (summing to 1)
impacts_str = '+,+,-,+,+'         # Example impacts: C1+, C2+, C3-, C4+, C5+
output_filename = 'result.csv'

topsis(input_filename, weights_str, impacts_str, output_filename)

print(f"TOPSIS calculation complete. Results saved to '{output_filename}'")

Success: TOPSIS results saved to 'result.csv'
TOPSIS calculation complete. Results saved to 'result.csv'


In [36]:
# To view the result, we can read the 'result.csv' file back into a DataFrame
import pandas as pd

df_result = pd.read_csv('result.csv')

print("TOPSIS Results:")
display(df_result)

TOPSIS Results:


,Alternative,Criterion1,Criterion2,Criterion3,Criterion4,Criterion5,Topsis Score,Rank
0,A1,25,2000,5,10000,7,0.217051,4
1,A2,40,3000,4,8000,6,0.456737,3
2,A3,60,2500,3,12000,8,0.675147,1
3,A4,30,3500,4,9000,5,0.479554,2
